In [1]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from datetime import datetime, timedelta
import plotly.graph_objects as go
import datetime
import pandas as pd
import tweepy
import csv
import os
import time
from dateutil import tz
import glob
import quandl as q
import re
import plotly.graph_objects as go
import numpy as np

quandl_api_key = os.environ['quandl_api_key']

# # # start - read in BTC data # # #
datasource_btcusd = "BITFINEX/BTCUSD.csv"
btcusd_data = pd.read_csv("coindata/{}".format(
    datasource_btcusd.replace("/", " ")), index_col=0)
btcusd_data.index = pd.to_datetime(btcusd_data.index)

most_recent_stored_btcusd_date = btcusd_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")


if most_recent_stored_btcusd_date != todays_date:
    # start - get data from quandl api
    data = q.get(datasource_btcusd.split(".")[0],   start_date=most_recent_stored_btcusd_date,
                    end_date='{}'.format(todays_date),
                    api_key=quandl_api_key)
    data.info()
    data["First"] = data.Last.shift(1)
    data.dropna()
    btcusd_data = pd.concat([btcusd_data, data])
    btcusd_data = btcusd_data.sort_index()
    # store current df with up-to-date values
    btcusd_data.to_csv('coindata/{}'.format(
        datasource_btcusd.replace("/", " ")), index=True)
        
# # # end - read in BTC data # # #

In [22]:
fed_assets_quandl_key = "FED/RESPPA_N_WW"
fed_assets_data = q.get(fed_assets_quandl_key, 
            api_key=quandl_api_key
            )
fed_assets_data = fed_assets_data.dropna()
fed_assets_data.info()
fed_assets_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 987 entries, 2002-12-18 to 2021-11-10
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   987 non-null    float64
dtypes: float64(1)
memory usage: 15.4 KB


,Value
Date,
2002-12-18,720761.0
2002-12-25,733136.0
2003-01-01,732202.0
2003-01-08,724902.0
2003-01-15,721325.0
...,...
2021-10-13,8480942.0
2021-10-20,8564943.0
2021-10-27,8556181.0


In [23]:
fed_assets_data = fed_assets_data.sort_index()
fed_assets_data

,Value
Date,
2002-12-18,720761.0
2002-12-25,733136.0
2003-01-01,732202.0
2003-01-08,724902.0
2003-01-15,721325.0
...,...
2021-10-13,8480942.0
2021-10-20,8564943.0
2021-10-27,8556181.0


In [27]:
# store current df with up-to-date values
fed_assets_data.to_csv('coindata/{}'.format(
    fed_assets_quandl_key.replace("/", " ")), index=True)

In [31]:
# merge fed and btc
btcusd_data_and_fed = pd.merge(btcusd_data, fed_assets_data, left_index=True, right_index=True, how='left')
btcusd_data_and_fed

,High,Low,Mid,Last,Bid,Ask,Volume,First,200movingaverage,350_movingaverage,111_movingaverage,Value
Date,,,,,,,,,,,,
2014-11-11,371.450000,360.74,366.325,366.250000,366.25,366.40,10223.053999,367.290000,366.250000,366.250000,366.250000,NaN
2014-11-12,436.000000,365.23,420.045,420.050000,420.04,420.05,100556.357159,366.250000,393.150000,393.150000,393.150000,4488895.0
2014-11-13,475.000000,381.56,419.240,419.250000,419.23,419.25,142734.302425,420.050000,401.850000,401.850000,401.850000,NaN
2014-11-14,431.860000,382.22,398.295,398.300000,398.29,398.30,64809.518015,419.250000,400.962500,400.962500,400.962500,NaN
2014-11-15,410.100000,368.00,378.035,378.000000,378.00,378.07,43478.728396,398.300000,396.370000,396.370000,396.370000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-10,68958.000000,62866.00,64752.500,64761.000000,64751.00,64754.00,12199.757272,67011.000000,43803.077523,43803.077523,50135.676564,8663117.0
2021-11-11,65541.000000,64079.00,64865.500,64861.000000,64865.00,64866.00,2828.863864,64761.000000,43934.657523,43934.657523,50421.964852,NaN
2021-11-12,65421.000000,62255.00,64027.500,64027.432184,64027.00,64028.00,5086.913300,64861.000000,44068.647329,44068.647329,50692.175953,NaN


In [37]:
btcusd_data_and_fed["Value"] = btcusd_data_and_fed["Value"].ffill()
btcusd_data_and_fed = btcusd_data_and_fed.dropna()

In [50]:
btcusd_data_and_fed["BTC_per_FedAssets"] = (btcusd_data_and_fed["Last"] / btcusd_data_and_fed["Value"]) *1000
btcusd_data_and_fed["350_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=350, min_periods=1).mean()
btcusd_data_and_fed["111_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=111, min_periods=1).mean()

In [9]:
import pandas as pd
fed_assets_quandl_key = "FED/RESPPA_N_WW"

fed_assets_data = pd.read_csv("coindata/{}".format(
    fed_assets_quandl_key.replace("/", " ")), index_col=0)
fed_assets_data.index = pd.to_datetime(fed_assets_data.index)

In [10]:
fed_assets_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

'2021-11-10'

In [17]:
from datetime import datetime, timedelta
import datetime
most_recent_stored_fed_assets_date = fed_assets_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

todays_date = datetime.date.today() - timedelta(days=7)
todays_date = todays_date.strftime("%Y-%m-%d")


In [18]:
most_recent_stored_fed_assets_date

'2021-11-10'

In [19]:
todays_date

'2021-11-08'

In [56]:
fig = go.Figure(
    data=go.Scatter(
        x=btcusd_data_and_fed.index,
        y=btcusd_data_and_fed['BTC_per_FedAssets'],
        name="BTC/FED",
        mode='lines',
        marker=dict(
            # size=16,
            color="red",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        )
)
)

fig.add_trace(go.Scatter(x=btcusd_data_and_fed.index, y=btcusd_data_and_fed["350_movingaverage_per_FedAssets"],
                    mode='lines',
                    name='350_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="gold"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data_and_fed.index, y=btcusd_data_and_fed["350_movingaverage_per_FedAssets"]*2,
                    mode='lines',
                    name='2*350_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data_and_fed["111_movingaverage_per_FedAssets"],
                    mode='lines',
                    name='111_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

# btcusd_data["111_movingaverage"]
fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)

fig

In [9]:
btcusd_data = btcusd_data.dropna()
btcusd_data["350_movingaverage"] = pd.Series.rolling(btcusd_data["Last"], window=350, min_periods=1).mean()
btcusd_data["111_movingaverage"] = pd.Series.rolling(btcusd_data["Last"], window=111, min_periods=1).mean()
btcusd_data

,High,Low,Mid,Last,Bid,Ask,Volume,First,200movingaverage,350_movingaverage,111_movingaverage
Date,,,,,,,,,,,
2014-11-11,371.450000,360.74,366.325,366.250000,366.25,366.40,10223.053999,367.290000,366.250000,366.250000,366.250000
2014-11-12,436.000000,365.23,420.045,420.050000,420.04,420.05,100556.357159,366.250000,393.150000,393.150000,393.150000
2014-11-13,475.000000,381.56,419.240,419.250000,419.23,419.25,142734.302425,420.050000,401.850000,401.850000,401.850000
2014-11-14,431.860000,382.22,398.295,398.300000,398.29,398.30,64809.518015,419.250000,400.962500,400.962500,400.962500
2014-11-15,410.100000,368.00,378.035,378.000000,378.00,378.07,43478.728396,398.300000,396.370000,396.370000,396.370000
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-10,68958.000000,62866.00,64752.500,64761.000000,64751.00,64754.00,12199.757272,67011.000000,43803.077523,43803.077523,50135.676564
2021-11-11,65541.000000,64079.00,64865.500,64861.000000,64865.00,64866.00,2828.863864,64761.000000,43934.657523,43934.657523,50421.964852
2021-11-12,65421.000000,62255.00,64027.500,64027.432184,64027.00,64028.00,5086.913300,64861.000000,44068.647329,44068.647329,50692.175953


In [15]:
fig = go.Figure(
    data=go.Scatter(
        x=btcusd_data.index,
        y=btcusd_data['Last'],
        mode='lines',
        marker=dict(
            # size=16,
            color="black",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        )
    )
)

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data["350_movingaverage"]*2,
                    mode='lines',
                    name='350_movingaverage',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data["111_movingaverage"],
                    mode='lines',
                    name='111_movingaverage',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

# btcusd_data["111_movingaverage"]
fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)

fig